# **Homework 3 - Michelin restaurants in Italy**

*Group#20*

- **Marco Zimmatore** - [zimmatore.1947442@studenti.uniroma1.it](mailto:zimmatore.1947442@studenti.uniroma1.it)
- **Gabriele Cabibbo** - [cabibbo.2196717@studenti.uniroma1.it](mailto:cabibbo.2196717@studenti.uniroma1.it)
- **Emre Yeşil** - [1emreyesil@gmail.com](mailto:1emreyesil@gmail.com)
- **Emanuele Iaccarino** - [emanueleiaccarino.ei@gmail.com](mailto:emanueleiaccarino.ei@gmail.com)

___

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
import contractions
from nltk.stem import WordNetLemmatizer
import re
import json
from collections import defaultdict
from DataCollection import engine

## **1. Data collection**

Although it would be ideal to call all functions together under ``` engine.py ``` and perform the parsing in a single step, this approach was not possible with our "more efficient" implementation. 

We’ll explore these issues later on, along with the solution implemented to address them.

Anyway I left ``` engine.py ``` file under DataColletion folder: the file works there but doesn't here on this Jupyter Notebook

### **1.1 Get the list of Michelin restaurants**

Fun Fact: I first did this part of the code 04/11/2024, the next day the number of restaurants was reduced from 2037 to 1983 (this is confirmed by the official site where now we have 100 pages instead of 102)

In [23]:
from DataCollection.crawler import get_michelin_urls
import logging
import time

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

start_time = time.time()
get_michelin_urls()
logging.info(f"Time to collect urls: {time.time() - start_time} seconds") # <3

DEBUG:root:Analyzing page 1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): guide.michelin.com:443
DEBUG:urllib3.connectionpool:https://guide.michelin.com:443 "GET /en/it/restaurants/page/1 HTTP/1.1" 200 None
DEBUG:root:Found restaurant URL: https://guide.michelin.com/en/campania/gragnano/restaurant/o-me-o-il-mare
DEBUG:root:Found restaurant URL: https://guide.michelin.com/en/abruzzo/popoli_1845563/restaurant/donevandro
DEBUG:root:Found restaurant URL: https://guide.michelin.com/en/piemonte/alba/restaurant/ape-vino-e-cucina
DEBUG:root:Found restaurant URL: https://guide.michelin.com/en/campania/sorrento/restaurant/da-bob-cook-fish
DEBUG:root:Found restaurant URL: https://guide.michelin.com/en/basilicata/matera/restaurant/da-mo
DEBUG:root:Found restaurant URL: https://guide.michelin.com/en/sardegna/cagliari/restaurant/sa-domu-sarda
DEBUG:root:Found restaurant URL: https://guide.michelin.com/en/sicilia/palermo/restaurant/charleston
DEBUG:root:Found restaurant URL: https:/

### **1.2 Crawl Michelin restaurant pages**

Using asyncio in the web scraping code allows to handle multiple HTML downloads concurrently, making the process faster and more efficient. Instead of waiting for each download to complete before starting the next (sequentially), the event loop jumps to the next available task when one is waiting for data, allowing to download multiple pages at the same time.

More info can be found [here](https://www.zenrows.com/blog/asynchronous-web-scraping-python#what-is-asynchronous-web-scraping), while the official Github repository is avaible at this [link](https://github.com/oxylabs/asynchronous-web-scraping-python)


 Apparently the reason why ``` engine.py ``` was working but calling here was not, is that jupyter run an event loop in the background that is not compatible with our asyncio library

In [ ]:
from DataCollection.crawler import download_html_async

# I was having the following error: asyncio.run() cannot be called from a running event loop 
# solution was found here: https://community.openai.com/t/error-when-using-langchain-webresearchretriever-runtimeerror-asyncio-run-cannot-be-called-from-a-running-event-loop/341969/5
# and here: https://github.com/googlecolab/colabtools/issues/3720
async def main(): # This allow us to run the function asynchronously
    await download_html_async() 

await main()

# Apparently the reason why engine.py was working but calling here was not, is that jupyter run an event loop in the background that is not compatible with our asyncio library

INFO:root:Downloading batch 1
DEBUG:root:Saved HTML to page_1\restaurant_16.html
DEBUG:root:Saved HTML to page_1\restaurant_5.html
DEBUG:root:Saved HTML to page_1\restaurant_17.html
DEBUG:root:Saved HTML to page_1\restaurant_10.html
DEBUG:root:Saved HTML to page_1\restaurant_11.html
DEBUG:root:Saved HTML to page_1\restaurant_18.html
DEBUG:root:Saved HTML to page_1\restaurant_6.html
DEBUG:root:Saved HTML to page_1\restaurant_20.html
DEBUG:root:Saved HTML to page_1\restaurant_8.html
DEBUG:root:Saved HTML to page_1\restaurant_7.html
DEBUG:root:Saved HTML to page_1\restaurant_4.html
DEBUG:root:Saved HTML to page_1\restaurant_13.html
DEBUG:root:Saved HTML to page_1\restaurant_19.html
DEBUG:root:Saved HTML to page_1\restaurant_9.html
DEBUG:root:Saved HTML to page_1\restaurant_12.html
DEBUG:root:Saved HTML to page_1\restaurant_3.html
DEBUG:root:Saved HTML to page_1\restaurant_15.html
DEBUG:root:Saved HTML to page_1\restaurant_1.html
DEBUG:root:Saved HTML to page_1\restaurant_2.html
DEBUG:root

In [ ]:
# moves folders under same main folder for cleaness
from DataCollection.organize_folders import organize_folders
organize_folders()

Created main folder: michelin_restaurants
Moved folder 'page_1' to 'michelin_restaurants'
Moved folder 'page_10' to 'michelin_restaurants'
Moved folder 'page_100' to 'michelin_restaurants'
Moved folder 'page_11' to 'michelin_restaurants'
Moved folder 'page_12' to 'michelin_restaurants'
Moved folder 'page_13' to 'michelin_restaurants'
Moved folder 'page_14' to 'michelin_restaurants'
Moved folder 'page_15' to 'michelin_restaurants'
Moved folder 'page_16' to 'michelin_restaurants'
Moved folder 'page_17' to 'michelin_restaurants'
Moved folder 'page_18' to 'michelin_restaurants'
Moved folder 'page_19' to 'michelin_restaurants'
Moved folder 'page_2' to 'michelin_restaurants'
Moved folder 'page_20' to 'michelin_restaurants'
Moved folder 'page_21' to 'michelin_restaurants'
Moved folder 'page_22' to 'michelin_restaurants'
Moved folder 'page_23' to 'michelin_restaurants'
Moved folder 'page_24' to 'michelin_restaurants'
Moved folder 'page_25' to 'michelin_restaurants'
Moved folder 'page_26' to 'm

### **1.3 Parse downloaded pages**

With the same method, we use asyncio for parsing, allowing to process multiple HTML files at the same time.
Additionally, the use of batches allow us to control the parallelism without overloading our sources, this way we analyze 100 files per batch. 

Smart and efficient way to perform it!

In [ ]:
from DataCollection.parser import parse_all_restaurants
import asyncio
await parse_all_restaurants()

INFO:root:Starting to parse all restaurants.
DEBUG:root:TSV file header written.
INFO:root:Processing batch 1 with 100 files.
DEBUG:root:Processing file: michelin_restaurants\page_1\restaurant_2.html, URL: https://guide.michelin.com/en/abruzzo/popoli_1845563/restaurant/donevandro
DEBUG:root:Processing file: michelin_restaurants\page_1\restaurant_13.html, URL: https://guide.michelin.com/en/emilia-romagna/noceto_1827072/restaurant/palazzo-utini
DEBUG:root:Processing file: michelin_restaurants\page_1\restaurant_10.html, URL: https://guide.michelin.com/en/campania/marina-di-casal-velino/restaurant/alessandro-feo
DEBUG:root:Processing file: michelin_restaurants\page_1\restaurant_16.html, URL: https://guide.michelin.com/en/toscana/castiglione-della-pescaia/restaurant/la-trattoria-enrico-bartolini
DEBUG:root:Processing file: michelin_restaurants\page_1\restaurant_1.html, URL: https://guide.michelin.com/en/campania/gragnano/restaurant/o-me-o-il-mare
DEBUG:root:Processing file: michelin_restaur

In [ ]:
import shutil
folder_path = "michelin_restaurants"
shutil.rmtree(folder_path)
print(f"The folder '{folder_path}' has been deleted successfully.")
# it's too heavy to push it and anyway we don't neeed it anymore 

The folder 'michelin_restaurants' has been deleted successfully.


In [ ]:
from DataCollection.file_type_converter import tsv_to_csv
tsv_to_csv('michelin_restaurants_data.tsv', 'michelin_restaurants_data.csv')

Conversion complete: michelin_restaurants_data.csv


## **2. Search Engine**

___

### **2.0 Preprocessing**

#### **2.0.1 Preprocessing the Text**

In [3]:
# Load The Datframe
df = pd.read_csv("michelin_restaurants_data.csv")

In [4]:
# Set up the stopwords list
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    if isinstance(text, str):


        words=text.split()
        words=[word for word in words if word.lower() not in stop_words]
        cleaned_text = ' '.join(words)

        return cleaned_text
    else:
            return ""  



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Marco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df['no_stopwords']=df['description'].apply(remove_stopwords)

In [6]:

# Set options to display all columns, rows, and no truncation
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [7]:
print(df[['description', 'no_stopwords']].head())


                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             description  \
0                                                                                                                           

In [8]:
def remove_punc(text):
    if isinstance (text,str):
         return text.translate(str.maketrans('', '', string.punctuation))
    else:
        return ""       

In [9]:
df['no_stopwords_and_punct'] = df['no_stopwords'].apply(remove_punc)

# Display the first few rows to verify the result
print(df[['description', 'no_stopwords', 'no_stopwords_and_punct']].head())

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             description  \
0                                                                                                                           

In [10]:
stemmer=PorterStemmer()

def apply_stemming(text):
    words= text.split()
    stemmed_words=[stemmer.stem(word) for word in words]
    stemmed_text=' '.join(stemmed_words)
    return stemmed_text

In [11]:
# Apply stemming to the 'no_stopwords_and_punct' column
df['no_stpwrd_punc_and_stemmed'] = df['no_stopwords_and_punct'].apply(apply_stemming)

# Display the first few rows to verify the result
print(df[['description', 'no_stopwords_and_punct', 'no_stpwrd_punc_and_stemmed']].head())

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             description  \
0                                                                                                                           

In [12]:
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def apply_lemmatization(text):
    text = text.lower()
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    lemmatized_text = ' '.join(lemmatized_words)  
    return lemmatized_text

df['lemmatized_text'] = df['no_stopwords_and_punct'].apply(apply_lemmatization)
print(df[['description', 'no_stopwords', 'no_stopwords_and_punct', 'lemmatized_text']].head())


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Marco\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             description  \
0                                                                                                                           

After I checked the results of Stemming  and lemmatizer, I prefer to use lemmatized's output. Because I realized that,Stemming caused same meaningless words. I think this method uses directly throw away same letters from end of the words. As a result, according to outputs of two methods I will contuinue with lemmatizer. (For example:restaurant>>>restaur)


In [13]:
# Removing Extra Whitespace

df['lemmatized_text+'] = df['lemmatized_text'].str.replace(r'\s+', ' ', regex=True).str.strip()


# Removing  Non-ASCII Characters or Emojis
df['cleaned_text'] = df['lemmatized_text+'].str.encode('ascii', 'ignore').str.decode('ascii')


In [14]:
#Expanding Contractions
df['cleaned_text'] = df['cleaned_text'].apply(lambda x: contractions.fix(x))


In [15]:
#Removing URLs and Email Addresses
df['cleaned_text'] = df['cleaned_text'].str.replace(r'http\S+|www\S+|https\S+', '', regex=True)
df['cleaned_text'] = df['cleaned_text'].str.replace(r'\S+@\S+', '', regex=True)
df['restaurant_name'] = df['restaurant_name'].str.split('–|-').str[0].str.strip()
df['restaurant_name'].head(5)

0                        O Me O Il Mare
1                        Alessandro Feo
2                                  Dama
3    Il Ristorante Alain Ducasse Napoli
4                         Palazzo Utini
Name: restaurant_name, dtype: object

### **2.1 Conjunctive Query** 

#### **2.1.1 Create Your Index!**

In [16]:
# Create a unique document ID for each restaurant entry in the DataFrame
df['document_id'] = df.index  


vocab = {}
current_term_id = 0
inverted_index = defaultdict(list)  

# Function to get or assign a term ID
def get_term_id(word):
    """Assigns a unique ID to each word in the vocabulary."""
    global current_term_id
    # If the word is not already in the vocabulary, add it with a new ID
    if word not in vocab:
        vocab[word] = current_term_id
        current_term_id += 1  
    return vocab[word]  

# Go through each restaurant's description and build the vocabulary and index
for _, row in df.iterrows():
    doc_id = row['document_id']
    words = row['cleaned_text'].split()  
    unique_words = set(words)  

    # For each unique word, get its term ID and add the document ID to the inverted index
    for word in unique_words:
        term_id = get_term_id(word)
        inverted_index[term_id].append(doc_id)

# Convert the vocabulary dictionary to a DataFrame and save it to a CSV file for future reference
vocab_df = pd.DataFrame(list(vocab.items()), columns=['word', 'term_id'])
vocab_df.to_csv('vocabulary.csv', index=False)

# Save the inverted index as a JSON file to use later when processing queries
with open('inverted_index.json', 'w') as f:
    json.dump(inverted_index, f)

# Load the inverted index and vocabulary when we need to process a search query
with open('inverted_index.json', 'r') as f:
    inverted_index = json.load(f)

# Reload vocabulary from CSV into a dictionary
vocab_df = pd.read_csv('vocabulary.csv')
vocab = dict(zip(vocab_df['word'], vocab_df['term_id']))

#### **2.1.2 Execute the Query**

In [17]:

# Save inverted index to JSON
with open('inverted_index.json', 'w') as f:
    json.dump(inverted_index, f)

# Load inverted index and vocabulary (for query processing)
with open('inverted_index.json', 'r') as f:
    inverted_index = json.load(f)

vocab_df = pd.read_csv('vocabulary.csv')
vocab = dict(zip(vocab_df['word'], vocab_df['term_id']))

# Query preprocessing function
def preprocess_query(query):
    """Processes a user query to remove punctuation and stop words, returning term IDs."""
    query = query.lower()
    query = re.sub(r'[^\w\s]', '', query)  # Remove punctuation
    query_terms = [word for word in query.split() if word not in stop_words] 
    # Convert each word to its term ID if it's in the vocabulary
    term_ids = [vocab.get(word) for word in query_terms if word in vocab]
    return term_ids  # Return the list of term IDs for the query

# %% Search Function
def search_restaurants(query):
    """Finds and returns restaurants that match all words in the query."""
    term_ids = preprocess_query(query)
    if not term_ids:
        print("No matching terms found in the vocabulary.")
        return pd.DataFrame()  # No matching terms

    # Get document IDs that contain all the query terms
    matching_docs = set(inverted_index.get(str(term_ids[0]), []))  
    for term_id in term_ids[1:]:
        matching_docs.intersection_update(inverted_index.get(str(term_id), [])) 

    # If there are matches, retrieve their restaurant details
    if matching_docs:
        results = df[df['document_id'].isin(matching_docs)][['restaurant_name', 'address', 'description', 'url']]
        return results.head(5)
    else:
        print("No documents match all query terms.")
        return pd.DataFrame()  # No matching documents



In [18]:
# Example usage
query = "moder cuisine restaurant"
results = search_restaurants(query)
print(results)

                       restaurant_name                    address  \
0                       O Me O Il Mare             Via Roma 45/47   
3   Il Ristorante Alain Ducasse Napoli  Via Cristoforo Colombo 45   
7        La Trattoria Enrico Bartolini           Località Badiola   
10                              Ménage        Via Euplio Reina 13   
11                          Donevandro            via Garibaldi 2   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

### **2.2 Ranked Search Engine with TF-IDF and Cosine Similarity**

#### **2.2.1 Inverted Index with TF-IDF Scores**

In [83]:
import importlib
importlib.reload(engine)


<module 'DataCollection.engine' from 'c:\\Users\\Marco\\Desktop\\Data Science Magistrale\\ADM\\ADM-HW3\\ADM-HW3\\DataCollection\\engine.py'>

1. **tfIdf Scores**: Calculate TF-IDF scores for each term in each restaurant’s description.


In [79]:
df_tfidf = df.copy()
file_path = "inverted_index.json"
with open(file_path, 'r', encoding='utf-8') as file:
    inverted_index_dict = json.load(file)
df_tfidf["description"] = df_tfidf["description"].fillna("")
updated_inverted_index = engine.compute_tfidf(df_tfidf, inverted_index_dict)


2. **Updated Inverted Index**: Build a new inverted index where each entry is a term, and the value is a list of tuples containing document IDs and TF-IDF scores.

In [80]:
# Finally we load the inverted_index in a JSON file
with open("updated_inverted_index.json", "w") as file:
    json.dump(updated_inverted_index, file)


#### **2.2.2 Execute the Ranked Query**


1. Process the query terms.

In [45]:
query = "MODERN; Seasonal CUISINE."

query_series = pd.Series(query)
query_series=query_series.apply(remove_stopwords).apply(apply_stemming).apply(remove_punc).apply(apply_lemmatization)
query_preprocessed = query_series.iloc[0]
print(query_preprocessed)





modern season cuisine


2. Use Cosine Similarity to rank matching restaurants based on the TF-IDF vectors of the query and each document.


In [84]:
vocabulary = pd.read_csv("vocabulary.csv")
similarities = engine.rank_documents(query_preprocessed, updated_inverted_index, df, vocabulary)
similarities

{3: [('162', 0.0), ('8', 0.0)], 6: [('162', 0.0)], 9: [('162', 0.0)], 11: [('162', 0.0), ('8', 0.0)], 19: [('162', 0.0)], 22: [('162', 0.0), ('8', 0.0)], 28: [('162', 0.0)], 36: [('162', 0.0), ('933', 0.0), ('8', 0.0)], 37: [('162', 0.0), ('8', 0.0)], 40: [('162', 0.0), ('8', 0.0)], 42: [('162', 0.0), ('8', 0.0)], 43: [('162', 0.0), ('8', 0.0)], 45: [('162', 0.0), ('933', 0.0), ('8', 0.0)], 55: [('162', 0.0)], 63: [('162', 0.0), ('8', 0.0)], 64: [('162', 0.0), ('8', 0.0)], 65: [('162', 0.0)], 70: [('162', 0.0), ('8', 0.0)], 75: [('162', 0.0), ('8', 0.0)], 79: [('162', 0.0), ('8', 0.005005823271337872)], 81: [('162', 0.0)], 88: [('162', 0.0), ('8', 0.0)], 91: [('162', 0.0)], 92: [('162', 0.0), ('8', 0.004657592087244803)], 95: [('162', 0.0)], 96: [('162', 0.0)], 98: [('162', 0.0), ('8', 0.0)], 105: [('162', 0.0), ('8', 0.0)], 107: [('162', 0.0), ('8', 0.0)], 116: [('162', 0.0), ('8', 0.0)], 120: [('162', 0.0), ('8', 0.0)], 123: [('162', 0.0), ('8', 0.0)], 129: [('162', 0.0), ('8', 0.0)]

[(79, 0.5934757746836791),
 (92, 0.5934757746836791),
 (413, 0.5934757746836791),
 (467, 0.5934757746836791),
 (879, 0.5934757746836791),
 (894, 0.5934757746836791),
 (903, 0.5934757746836791),
 (1122, 0.5934757746836791),
 (1616, 0.5934757746836791),
 (1689, 0.5934757746836791),
 (1863, 0.5934757746836791),
 (564, 0.5934757746836791),
 (719, 0.5934757746836791),
 (754, 0.5934757746836791),
 (1776, 0.5934757746836791),
 (224, 0.5934757746836791),
 (289, 0.5934757746836791),
 (348, 0.5934757746836791),
 (466, 0.5934757746836791),
 (483, 0.5934757746836791),
 (528, 0.5934757746836791),
 (617, 0.5934757746836791),
 (710, 0.5934757746836791),
 (793, 0.5934757746836791),
 (852, 0.5934757746836791),
 (952, 0.5934757746836791),
 (964, 0.5934757746836791),
 (1041, 0.5934757746836791),
 (1043, 0.5934757746836791),
 (1052, 0.5934757746836791),
 (1223, 0.5934757746836791),
 (1256, 0.5934757746836791),
 (1459, 0.5934757746836791),
 (1530, 0.5934757746836791),
 (1611, 0.5934757746836791),
 (1638, 0

3. Return the top-k results or all matching restaurants if fewer than k have non-zero similarity.

In [96]:
k = 5  
top_k_values = similarities[:k]
print(top_k_values)
documents_best_k = df_tfidf.iloc[[top_k_index[0] for top_k_index in top_k_values]]
documents_best_k["similarity_score"] = [values[1] for values in top_k_values]
documents_best_k[["restaurant_name", "address", "description", "url", "similarity_score"]]

[(79, 0.5934757746836791), (92, 0.5934757746836791), (413, 0.5934757746836791), (467, 0.5934757746836791), (879, 0.5934757746836791)]


C:\Users\Marco\AppData\Local\Temp\ipykernel_38156\3078987059.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  documents_best_k["similarity_score"] = [values[1] for values in top_k_values]


,restaurant_name,address,description,url,similarity_score
79,Giglio,piazza del Giglio 2,"Housed in a beautiful 18C palazzo, Giglio was founded in 1979 and is now run by three young men who took over just a few years ago. These three friends form a real team, who have come together to inject new life into the restaurant which stands on one of the many squares in Lucca’s charming historic centre. In fine weather, guests can enjoy the lively summer atmosphere outdoors, while those who prefer a quieter ambience can book a table in the more austere and elegant dining room indoors. Modern cuisine with a very Italian feel, combined with French techniques and the occasional use of international ingredients.",https://www.ristorantegiglio.com/,0.593476
92,Osteria Arborina,frazione Annunziata 27/b,"Although the beautiful Arborina Relais boasts modern, stylish and elegant dining rooms, the main attraction here is the restaurant’s exciting cuisine in which different culinary styles and techniques are showcased in every dish. Inspiration comes from different influences – for example, we chose a menu inspired by the 1980s (a period when Neapolitan chef Fernando Tommaso Forino was not yet born), and particularly enjoyed the mullet in vodka and the dessert trolley. We also liked the idea behind the menu as a whole, namely the no-waste focus, with all parts of the ingredients used yet always with the aim of creating enjoyable dishes. An interesting, classically extensive wine list and excellent service complete the picture.",https://arborinarelais.it/osteria/,0.593476
413,Manuelina,via Roma 296,"Very few restaurants in Liguria can compete with the long gastronomic tradition of this restaurant, which was named after its original owner back in 1885! It was Manuelina herself who made the typical focaccia of Recco famous, offering it as a dish in her tavern. The fourth generation of the family now runs the business with the same passion and enthusiasm – part of the building houses the typical Focacceria, where family traditions are kept alive, while the other is home to the restaurant itself (even here the legendary and highly recommended focaccia is on the menu) with its round tables. The cuisine is traditional yet with modern touches and based on Ligurian ingredients with a focus on fish (although meat also features).",https://www.manuelinaristorante.it/,0.593476
467,Benso,piazza Cavour 7,"Situated in a small garden in the historic centre, this modern bistro has a light, airy feel thanks to its large windows and its outdoor tables perfect for alfresco dining in summer. The young kitchen staff, supervised by a talented chef and with plenty of experience behind them, serve contemporary cuisine with an individual flavour. Simple aperitif-style options are available between 6 and 8pm.",https://www.bensofood.com/,0.593476
879,Maso Palù,via Graziani 56,"An annexe to the family farm with its organic apple orchards, this rustic, welcoming restaurant situated on the road leading up towards Monte Baldo has been in business since 1984. Despite its years of experience, the management team here does not rest on its laurels but continues to serve cuisine rooted in regional traditions with just a touch of imagination and modern influence in some dishes. Open every day in summer, but only Friday to Sunday at other times of year.",http://www.masopalu.com,0.593476


## **3. Define a New Score!**

___

1. **User Query**: The user provides a text query. We’ll retrieve relevant documents using the search engine built in Step 2.1.

2. **New Ranking Metric**: After retrieving relevant documents, we’ll rank them using a new custom score. Instead of limiting the scoring to only the description field, we can include other attributes like `priceRange`, `facilitiesServices`, and `cuisineType`.

3. You will use a **heap data structure** (e.g., Python’s heapq library) to maintain the *top-k* restaurants.


## **4. Visualizing the Most Relevant Restaurants**

___

## **5. BONUS: Advanced Search Engine**

___

## **Algorithmic Question (AQ)**
___

1. Write the pseudocode for an algorithm that solves this problem.


### **Overview**
The algorithm is designed to guide a robot that collects packages located on a grid, starting from the origin point (0,0). The robot can only move right ('R') or up ('U'). The goal is to collect all packages using the fewest moves while maintaining a lexicographically smallest path.


#### **Input**
- The number of test cases, `t`.
- For each test case:
  - The number of packages, `n`.
  - The coordinates `(x_i, y_i)` of each package.

#### **Output**
- For each test case, print "YES" followed by the smallest path if all packages can be collected, or "NO" if it is impossible.


### **Pseudocode**
- The idea behind the followed *pseudocode* is that we want to divide each problem into smaller problems. Instead of thinking about "How can we collect all the packages in the best way?", we can firstly solve the problem "How can i reach in the fastes way the next package?" and iterate over the packages.
- To develop this in the most efficient way, we have to sort the collection that will contain the coordinates of each package we are receiving in input, because we dont want to solve in the brute force way trying every path possible. To do that
we compute the distance between the origin and each of the coordinates and we sort them on this distance. 
- Then we compute the **Manhattan Distance** (or **L1 distance**) that helps up to compute the number of single steps from point A to point B:
$$ d = |x_1 - x_2| + |y_1 - y_2| $$
- We iterate over this number and in each iteration we decide to go up or right giving more value to the right because lexicographycally  $"R" < "U"$
- Finally return the Path scaling this idea

Here the pseudocode:

```python
    FOR each test_case FROM 1 TO t DO
      n = READ integer from input
      list_of_coordinates = []

      FOR each package FROM 1 to N DO
        coordinates = READ integers from input
        ADD coordinates to list_of_coordinates

      sorted_coordinates = sort(list_of_coordinates)

      FOR target_x, target_yy IN sorted_coordinates DO
        if target_x or target_y LESS THAN current_coordinates
          BREAK
          PRINT("NO")

        COMPUTE DISTANCE BETWEEN target_x, target_y AND current_coordinates

        FOR steps FROM 1 TO distance DO
          IF current_coordinate_x LESS THAN target_x 
            THEN MOVE RIGHT AND ADD "R" TO PATH
            CONTINUE

          ELSE IF current_coordinate_Y LESS THAN target_y
            THEN MOVE UP AND ADD "U" TO PATH

    RETURN PATH


```




Then we have implemented in python this solution using numpy package to make the sorting more efficient:

In [ ]:
import numpy as np 
import functions

with open("algorithmic_input.txt") as file:

    # Select the number of Test Cases
    t = int(file.readline().strip())
    # Iterate over the number fo test cases
    for _ in range(t):
        
        # Extract the number of Packages to collect
        number_packages = int(file.readline().strip())

        # Store all the input coordinate into the Numpy Array
        list_of_coordinates = np.array([tuple(map(int, file.readline().strip().split())) for _ in range(number_packages)] , dtype = int)

        # I compute the distance from the (0,0) point to sort the list
        distances = np.sqrt(list_of_coordinates[:, 0]**2 + list_of_coordinates[:, 1]**2)

        # I have used the argsort function in order to return the indices of the distances ordered in ascendent way
        sorted_indices = np.argsort(distances)

        # I return the list ordered
        sorted_coordinates = list_of_coordinates[sorted_indices]

        # Execute the function to return the smallest path given an array of sorted coordinates
        functions.compute_smallest_path(sorted_coordinates)

    




YES
RUUURRRRUU
NO
YES
RRRRUUU


2. Prove that your algorithm is correct.

To prove the correctness of our algorithm, which contains a loop, the best way to do it is applying the **Invariant of the Loop**.
It is a logic condition that remains true at the beginning and end of each iteration of a loop during the execution of an algorithm.
Showing that a loop has an invariant can help to prove that the loop behaves correctly and in general the correctness of the algorithm.

#### **Invariant of the Loop**

**Loop Invariant**: At the beginning of each iteration, the current_coordinate represents a point reachable from the origin using only "U" and "R" moves, and result contains a valid path from the origin to current_coordinate.

**Proof**:

- **Initialization**: The starting point is (0, 0), which is trivially reachable from the origin.

- **Maintenance**:
The algorithm checks if the next coordinate (`target_x`, `target_y`) can be reached using only "U" and "R". This is verified by checking that `target_x` is not less than `current_coordinate[0]` and `target_y` is not less than `current_coordinate[1]`.
If this condition fails, the algorithm outputs "NO" and terminates, proving that reaching all points in the list is impossible with the allowed moves.
If the condition holds, the algorithm updates current_coordinate and appends "R" and "U" moves accordingly, ensuring that only valid moves are added to result.

- **Termination**: The loop iterates over all coordinates in list. When it completes without triggering the "NO" condition, it implies that all coordinates were reachable from the previous position using only "U" and "R" moves.

**Conclusion**: The loop invariant holds true throughout each iteration, guaranteeing that result represents a valid path up to `current_coordinate`.

Obviously we can scale up the invariant also for the external loop over the number_of_packages to complete the proof.

___
3. Compute the time complexity of your algorithm in Big O notation. Break down the steps involved in the algorithm, and explain which parts contribute most to the overall time complexity.
___

To Compute the time complexity of the algorithm int the **BIG O** Notation we have to consider in the *worst case*  the operation in your algorithm that consumes the most execution time.

The *worst case* happens when we are able to collect all the packages in input, so there is no stopping condition.

Let's break down the steps in the algorithm.
We define $t$ as the number of `test cases` and $n$ as the number of `packages` our robot has to collect each test case $t$.

- So at first we have to execute a for loop to iterate over the `test cases` t, so everything we'll be describe below will be executed $t$ times:
    ```python
                            for _ in range(t):
    ```
    - Then we use a list comprehension (*python functionality*) to store all the coordinates of the `packages` for that test case:
    ```python
    list_of_coordinates = np.array([tuple(map(int, file.readline().strip().split())) for _ in range(number_packages)] ,dtype = int)
    ```
    - The previous instruction maps a series of $n$ tuples (`number of packages`) inside the numpy array, so the instruction 
    ```python 
                            tuple(map(int, file.readline().strip().split())) 
    ``` 
    - will be executed $n$ times.

    - The **fundamental instruction** of the algorithm, is:
    ```python 
                            sorted_indices = np.argsort(distances)
    ```
    - Infact, as we know in the worst case the most efficient **Sorting algorithms** based on comparisons like *Merge Sort* have a lower bound of $O(n\log(n))$

Even if the previous instruction consumes the most execution time, let's be specific in the calculus of complexity and dive in the remaining part of the code.

- the function `compute_smallest_path` takes in input the *list_of_sorted_coordinates* and outputs the smallest path to collect all the packages if it is possible:


    - Ignoring instructions of initialization we consider $O(1)$ computationally, we iterate over the input list of size $n$ that contains the coordinates of each package sorted by the distance to the origin:

```python 
                            for target_x, target_y in list:
``` 

- The *fundamental operation* in this function is this loop, that in the worst case will be executed $s_i$ times where $s_i$ is the single step distance between the i-th and i+1-th package. 

```python 
                            for _ in range(distance_from_next):

``` 

- If we call $p$ the number of total steps, $p =  \sum_{i=1}^{n} s_i $


So the algorithm will have time complexity equal to $O(t \times n\log(n)) + O(t \times p \times n)$



___
4. Ask an LLM tool (such as ChatGPT, Claude AI, Gemini, Perplexity, etc.) to evaluate the time complexity of your code using Big O notation. Is the assessment accurate? If it differs from your previous analysis, which would be correct? Please explain your reasoning.
___

> My prompt to *ChatGpt* was `"##CODE## evaluate the time complexity of this code using Big O notation "`.

This was the answer:

To analyze the time complexity of this code, let’s break down each section and determine the number of operations relative to $n$, where $n$ represents the number of packages.

#### 1. **Reading and Parsing Input**
   - **File Reading**: The code reads $t$ test cases from a file. For each test case, it reads the number of packages and coordinates.
   - **Parsing Coordinates**: Each coordinate pair is read and converted into an integer tuple, which is stored in a Numpy array. Parsing takes $O(n)$ time per test case since it reads $n$ coordinates, each in $O(1)$ time.
   
   **Time Complexity for Parsing Input**: $O(t \times n)$.

#### 2. **Calculating Distances**
   - For each coordinate, the distance from the origin $0,0$ is calculated using the Euclidean formula $\sqrt{x^2 + y^2}$.
   - Calculating the distance for each of the $n$ points takes $O(n)$ time, as computing the distance for each point is $O(1)$.

   **Time Complexity for Distance Calculation**: $O(t \times n)$.

#### 3. **Sorting Coordinates by Distance**
   - The code sorts the indices based on their distances from the origin. Sorting an array of $n$ elements takes $O(n \log n)$.
   - After sorting, the code creates a new array `sorted_coordinates` by rearranging the coordinates based on the sorted order, which is $O(n)$.

   **Time Complexity for Sorting**: $O(t \times n \log n)$.

#### 4. **Computing the Smallest Path**
   - The `compute_smallest_path` function iterates through each of the sorted coordinates in $O(n)$ time.
   - For each coordinate, it checks if the target coordinate can be reached by only moving "up" (U) or "right" (R). This involves a constant-time check and, if infeasible, it returns immediately.
   - If the target can be reached, the code calculates the distance along $x$ and $y$ axes.
   - For each coordinate, it adds the steps to reach it. Moving each step and updating coordinates takes $O(1)$, and each package requires a bounded number of steps (no more than $x + y$ operations per package).

   Overall, this is an $O(n)$ operation since it iterates through all coordinates once and performs a bounded amount of work for each coordinate.

   **Time Complexity for Smallest Path Calculation**: $O(t \times n)$.

#### 5. **Printing Results**
   - Printing “YES” or “NO” and the path string is $O(n)$ for each test case.

   **Time Complexity for Printing**: $O(t \times n)$.

#### **Total Time Complexity**

Summing up the complexities for each step, the dominant term is $O(t \times n \log n)$ from the sorting step. Thus, the overall time complexity is:

$$O(t \times n \log n)$$


where:
- $t$ is the number of test cases,
- $n$ is the number of packages per test case.

This complexity indicates that the code's performance primarily depends on the sorting operation within each test case.


 - I dont fully agree with the response give by *ChatGPT*. It has broken in small parts the calculus of time complexity like we have done, but it wasnt accurate in my opinion on the study of the algorithm `compute_smallest_path` because it has made the assumption that the size of the path $p$ is $O(1)$. 

- But if $p$ is $O(n)$ for example the **Total time complexity** is not $O(t \times \log(n))$ but it would be $O(t \times n^2)$ which would be the worst case because the function `compute_smallest_path` would consume the most amount of time.

5. Assume now that the robot can also move towards the left or downwards, and consider the greedy approach: from the current location go to the closest package. Notice that now we can always collect all packages. Prove that the greedy algorithm is optimal (i.e., it minimizes the total distance traveled), or provide a counterexample showing that it is not.   

The proof of optimality for this greedy approach relies on the optimal substructure of the problem. Specifically, we show that if the greedy choice is not optimal at some step, swapping the greedy move with an alternative will equalize or increase the total distance traveled. 

#### **Step-by-Step Analysis**: 

The algorithm is based on the calculus of the **Manhattan Distance** that minimizes by definition the number of single steps between two points.

**Initial State**: The robot starts at some position $(𝑥_0,𝑦_0)$ and is tasked with collecting a set of packages, each located at $( 𝑥_𝑖 , 𝑦_𝑖 )$.

**Greedy Step**: At each step, the robot moves to the closest package (because we have previously sorted the list of packages). 
Let’s assume the greedy algorithm at some stage goes from the current position $(x_k,y_k)$ to some package $( 𝑥_𝑞 , 𝑦_𝑞)$, and then from $(x_q ​ ,y_q ​ )$ to $(x_p ​ ,y_p ​ )$ selecting the closest package.

 Now, let’s consider an another algorithm alternative (not *greedy*) swapping the two steps: first move from $(x_k ​ ,y_k ​ )$ to $(x_p ​ ,y_p ​ )$, and then move from $( 𝑥_𝑝 , 𝑦_𝑝 )$ to $(x_q ​ ,y_q ​)$. 

 The total distance (equal to the **Manhattan Distance**) for the greedy choice is: $𝐷_{greedy} = |x_k − 𝑥_𝑝|+ |𝑦_𝑘 − 𝑦_𝑝| + |𝑥_𝑝 − 𝑥_𝑞| + |𝑦_𝑝 − 𝑦_𝑞|$

The total distance for the alternative choice is: $D_{alternative} ​ = ∣x_k ​− x_q ∣ + ∣y_k− y_q∣ + ∣x_q − x_p∣ + |y_q​ − y_p|$

The key observation here is that, by the triangle **inequality for Manhattan distance**, we have: 
$$ |x_k ​ −x_p ∣ + ∣y_k ​− y_p∣ + |𝑥_𝑝 − 𝑥_𝑞| + |𝑦_𝑝 − 𝑦_𝑞| \le ∣ x_k − x_q∣ + ∣y_k​ − y_q∣ + ∣x_q − x_p∣ + ∣y_q​ − y_p∣$$
Hence, moving directly from  $(x_k,y_k )$ to $(x_p,y_p)$ and then to $(x_q ,y_q)$ will not result in a greater distance than the alternative move. 
 
Therefore, the greedy approach (choosing the closest package at each step) is at least as good as any alternative strategy in terms of total distance.

For completeness, we have implemented also the function that simulates this extended case.

In [ ]:
import numpy as np 
import functions

with open("algorithmic_input.txt") as file:

    # Select the number of Test Cases
    t = int(file.readline().strip())
    # Iterate over the number fo test cases
    for _ in range(t):
        
        # Extract the number of Packages to collect
        number_packages = int(file.readline().strip())

        # Store all the input coordinate into the Numpy Array
        list_of_coordinates = np.array([tuple(map(int, file.readline().strip().split())) for _ in range(number_packages)] , dtype = int)

        # I compute the distance from the (0,0) point to sort the list
        distances = np.sqrt(list_of_coordinates[:, 0]**2 + list_of_coordinates[:, 1]**2)

        # I have used the argsort function in order to return the indices of the distances ordered in ascendent way
        sorted_indices = np.argsort(distances)

        # I return the list ordered
        sorted_coordinates = list_of_coordinates[sorted_indices]

        # Execute the function to return the smallest path given an array of sorted coordinates
        functions.extended_compute_smallest_path(sorted_coordinates)

    




YES
RUUURRRRUU
YES
RLU
YES
RRRRUUU


We can see that in this case we are able to collect all the packages, so "NO" is not printed in the output